In [1]:
import os
import time

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

SAVING_DIR='/home/data/taxonomy/'
os.environ["TRANSFORMERS_CACHE"] = SAVING_DIR + "hf_cache/"
os.environ["HF_HOME"] = SAVING_DIR + "hf_cache/"

import torch
from transformers_modified.src.transformers import AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer, AutoConfig, PretrainedConfig, AutoTokenizer
import math
import re
from operator import attrgetter
import torch.nn as nn
import torch.nn.functional as F
import transformers_modified.src.transformers as transformers

from typing import Optional, Dict, Sequence
from argparse import ArgumentParser
from pathlib import Path

from tqdm import tqdm
from dataclasses import dataclass, field
from itertools import chain
from functools import partial

import numpy as np
import torch
from tqdm import tqdm
from peft import PeftModel, get_peft_model, TaskType, LoraConfig

from ste_utils import prepare_llama_ste
import datasets
from datasets import load_dataset, load_from_disk

#import transformers
from transformers_modified.src.transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    default_data_collator,
    LlamaForCausalLM,
    TrainerCallback,
    DataCollatorForSeq2Seq
)
import transformers_modified.src.transformers as transformers


IGNORE_INDEX = -100

/home/data/LLM_Compression/transformers_modified/src/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/data/LLM_Compression/transformers_modified/src/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The model checkpoint for weights initialization. Don't set if you want to train a model from scratch."
            )
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=False,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    token: str = field(
        default=None,
        metadata={
            "help": (
                "The token to use as HTTP bearer authorization for remote files. If not specified, will use the token "
                "generated when running `huggingface-cli login` (stored in `~/.huggingface`)."
            )
        },
    )
    trust_remote_code: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether or not to allow for custom models defined on the Hub in their own modeling files. This option"
                "should only be set to `True` for repositories you trust and in which you have read the code, as it will "
                "execute code present on the Hub on your local machine."
            )
        },
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype. If `auto` is passed, the "
                "dtype will be automatically derived from the model's weights."
            ),
            "choices": ["auto", "bfloat16", "float16", "float32"],
        },
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    streaming: bool = field(default=False, metadata={"help": "Enable streaming mode"})
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Optional input sequence length after tokenization. "
                "The training dataset will be truncated in block of this size for training. "
                "Default to the model max input length for single sentence inputs (take into account special tokens)."
            )
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )

    dataset_percentage: Optional[int] = field(
        default=100,
        metadata={"help": "The number of percentage to take from entire dataset"},
    
    )
    
    seed: Optional[int] = field(
        default=42,
    )
    
    load_from_disk: bool = field(
        default=False
    )

@dataclass
class DataCollatorWithMaskForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, batch):
        input_ids = []
        labels = []
        attention_masks = []

        for item_dict in batch:
            input_ids.append(torch.tensor(item_dict["input_ids"]))
            attention_masks.append(torch.tensor(item_dict["attention_mask"]))
            label = torch.tensor(item_dict["labels"])
            label[:-1] = IGNORE_INDEX
            labels.append(label)

        input_ids = torch.vstack(input_ids)
        attention_masks = torch.vstack(attention_masks)
        labels = torch.vstack(labels)
            
        data_dict = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
        }
        if labels is not None:
            data_dict['labels'] = labels
        return data_dict
    



In [3]:
dataset_name = "allenai/tulu-v2-sft-mixture" #"wikitext" #'/home/LLM_Compression/logs/wikitext_gpt2'
dataset_config_name = None#'wikitext-2-raw-v1'
valid_split = 10
block_size = 32
dataset_percentage = 0.1

data_args = DataTrainingArguments(
    dataset_name = dataset_name,
    dataset_config_name = dataset_config_name,
    validation_split_percentage = valid_split,
    block_size = block_size,
    dataset_percentage = dataset_percentage
)


In [4]:
model_name_or_path = 'gpt2'
cache_dir = SAVING_DIR + "hf_cache/"

model_args = ModelArguments(
    model_name_or_path = model_name_or_path,
    config_name = None, 
    tokenizer_name = None,
    use_fast_tokenizer = True,
    token = None,
    trust_remote_code = True,
    cache_dir= cache_dir
)


In [5]:
 model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        torch_dtype=torch.bfloat16,
        cache_dir=model_args.cache_dir,
        device_map="auto"	    
)

In [6]:

# Load pretrained tokenizer
tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "token": model_args.token,
    "trust_remote_code": model_args.trust_remote_code,
}

if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)

In [7]:
from data_utils import encode_with_prompt_completion_format, encode_with_messages_format, load_hf_datasets

In [8]:
raw_datasets = load_hf_datasets(data_args)

In [9]:
if (
    "prompt" in raw_datasets["train"].column_names
    and "completion" in raw_datasets["train"].column_names
):
    encode_function = partial(
        encode_with_prompt_completion_format,
        tokenizer=tokenizer,
        max_seq_length=data_args.block_size,
    )
elif "messages" in raw_datasets["train"].column_names:
    encode_function = partial(
        encode_with_messages_format,
        tokenizer=tokenizer,
        max_seq_length=data_args.block_size,
    )

lm_datasets = raw_datasets.map(
    encode_function,
    batched=False,
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=[
        name
        for name in raw_datasets["train"].column_names
        if name not in ["input_ids", "labels", "attention_mask"]
    ],
    desc="Tokenizing and reformatting instruction data",
)

lm_datasets.set_format(type="pt")
lm_datasets = lm_datasets.filter(
    lambda example: (example["labels"] != -100).any()
)


Tokenizing and reformatting instruction data:   0%|          | 0/33 [00:00<?, ? examples/s]

Filter:   0%|          | 0/33 [00:00<?, ? examples/s]

In [10]:
labels = lm_datasets['train'][0]['labels']

In [11]:
lm_datasets['train'][0]

{'input_ids': tensor([   27,    91,  7220,    91,    29,   198, 16773,   510,   351,   257,
          1178,  7124,  4213,   329,   257,   649,  7072,    13,   198,    27,
            91,   562, 10167,    91,    29,   198,    16,    13,  5483,  2056,
          7124,    25]),
 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100,   16,   13, 5483, 2056, 7124,   25]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1])}

In [48]:
mask = labels != -100
labels[]

tensor([  16,   13, 5483, 2056, 7124,   25])

In [12]:
tokenizer.decode(lm_datasets['train'][0]['input_ids'])

'<|user|>\nCome up with a few marketing ideas for a new restaurant.\n<|assistant|>\n1. Social media marketing:'

In [43]:
tokenizer.decode(lm_datasets['train'][0]['labels'][-6:], skip_special_tokens=True)

'1. Social media marketing:'

In [34]:
raw_datasets['train'][0]

{'dataset': 'wizardlm',
 'id': 'wizardlm_alpaca_65567',
 'messages': [{'role': 'user',
   'content': 'Come up with a few marketing ideas for a new restaurant.'},
  {'role': 'assistant',
   'content': "1. Social media marketing: Utilize platforms like Facebook, Instagram, and Twitter to promote the restaurant's specials, menus, and ambiance.\n\n2. Partnership with local food bloggers: Invite local food bloggers to review the restaurant and feature it on their website or social media accounts, which can help to build awareness and attract new customers.\n\n3. Loyalty program: Establish a loyalty program with offers such as free drinks, discounts, and exclusive promotions to encourage repeat business.\n\n4. Special offers: Experiment with running special promotions, such as happy hour deals, promotions for families, and meal deals, to attract new customers and keep existing ones coming back.\n\n5. Create buzz: Hold a grand opening event to generate excitement and attract local media atten

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model, padding="longest"
)

In [7]:
raw_datasets = ld = load_hf_datasets(data_args)

In [16]:
encode_function = partial(
    encode_with_messages_format,
    tokenizer=tokenizer,
    max_seq_length=data_args.block_size,
)

lm_datasets = raw_datasets.map(
    encode_function,
    batched=False,
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=[
        name
        for name in raw_datasets["train"].column_names
        if name not in ["input_ids", "labels", "attention_mask"]
    ],
    desc="Tokenizing and reformatting instruction data",
)

lm_datasets.set_format(type="pt")
lm_datasets = lm_datasets.filter(
    lambda example: (example["labels"] != -100).any()
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model, padding="longest"
)


TypeError: encode_with_messages_format() missing 2 required positional arguments: 'tokenizer' and 'max_seq_length'

In [8]:
@dataclass
class DistillDataCollatorWithMaskForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, batch):
        input_ids = []
        labels = []
        attention_masks = []
        logits = []

        for item_dict in batch:
            input_ids.append(torch.tensor(item_dict["input_ids"]))
            attention_masks.append(torch.tensor(item_dict["attention_mask"]))
            label = torch.tensor(item_dict["labels"])
            label[:-1] = IGNORE_INDEX
            labels.append(label)
            logits.append(torch.tensor(item_dict['logits']).unsqueeze(0))

        input_ids = torch.vstack(input_ids)
        attention_masks = torch.vstack(attention_masks)
        labels = torch.vstack(labels)
        logits = torch.vstack(logits)
            
        data_dict = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'teacher_logits': logits
        }
        if labels is not None:
            data_dict['labels'] = labels
        return data_dict
    

In [9]:
data_collator = DistillDataCollatorWithMaskForCausalLM(tokenizer)
#data_collator = DataCollatorWithMaskForCausalLM(tokenizer)

In [10]:
training_args = TrainingArguments(
    output_dir = './test_distill',
    learning_rate = 3e-4, 
    seed = 2, 
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 1,
    gradient_checkpointing=False,
    save_strategy = 'steps',
    save_steps = 1000,
    evaluation_strategy = 'steps',
    eval_steps = 2,
    weight_decay = 0.1,
    warmup_ratio = 0.03,
    lr_scheduler_type = "cosine",
    logging_steps = 1,
    do_train = True,
    do_eval = True,
    report_to="wandb",
    remove_unused_columns=False
)

train_dataset = ld["train"]
eval_dataset = ld["validation"]

class MegaTrainer(Trainer):
    def __init__(self, model, temperature=None, lambda_param=None,  *args, **kwargs):
        super().__init__(model=model, *args, **kwargs)
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.temperature = temperature
        self.lambda_param = lambda_param
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
            label_mask = labels != -100
        else:
            labels = None

        logits = inputs.pop("teacher_logits")[label_mask]

        outputs = model(**inputs)

        #https://huggingface.co/docs/transformers/tasks/knowledge_distillation_for_image_classification
        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(outputs.logits[label_mask] / self.temperature, dim=-1)

        # Compute the loss
        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        print(distillation_loss)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            unwrapped_model = unwrap_model(model)
            if _is_peft_model(unwrapped_model):
                model_name = unwrapped_model.base_model.model._get_name()
            else:
                model_name = unwrapped_model._get_name()
            if model_name in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                student_target_loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                student_target_loss = self.label_smoother(outputs, labels)
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            student_target_loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        self.state.distill_loss = distillation_loss.detach().cpu().item()
        self.state.CE_loss = student_target_loss.detach().cpu().item()
        # Calculate final loss
        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, outputs) if return_outputs else loss

    def log(self, logs: Dict[str, float]) -> None:
        """
        Log `logs` on the various objects watching training.

        Subclass and override this method to inject custom behavior.

        Args:
            logs (`Dict[str, float]`):
                The values to log.
        """
        if self.state.epoch is not None:
            logs["epoch"] = round(self.state.epoch, 2)
        if self.args.include_num_input_tokens_seen:
            logs["num_input_tokens_seen"] = self.state.num_input_tokens_seen

        #print(self.state)
        if (self.state.global_step % self.state.eval_steps) == 0:
            #print(self.state.global_step, self.state.logging_steps, (self.state.global_step % self.state.logging_steps) == 0)
            prefix = 'eval_'
        else:
            prefix = 'train_'
        logs[f'{prefix}distill_loss'] = self.state.distill_loss
        logs[f'{prefix}CE_loss'] = self.state.CE_loss

        output = {**logs, **{"step": self.state.global_step}}
        self.state.log_history.append(output)
        self.control = self.callback_handler.on_log(self.args, self.state, self.control, logs)


class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        #print(logs)
        pass
    
# Initialize our Trainer
trainer = MegaTrainer(
    model=model,
    args=training_args,
    train_dataset=ld['train'],
    eval_dataset=ld['validation'],
    tokenizer=tokenizer,
    # Data collator will default to DataCollatorWithPadding, so we change it.
    data_collator=data_collator, 
    temperature=1, 
    lambda_param=1
)
trainer.add_callback(PrinterCallback)

In [11]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vityavitalich. Use `wandb login --relogin` to force relogin


tensor(15.7120, device='cuda:0', grad_fn=<MulBackward0>)


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Distill Loss,Ce Loss
2,13.809000,4.361210,4.588900,1.308000,0.654000,5.810260,0.925781
4,15.695900,11.265380,4.679600,1.282000,0.641000,13.158327,0.878906
6,14.890100,8.974860,4.574400,1.312000,0.656000,13.189489,0.820312
8,10.545400,7.619331,4.586300,1.308000,0.654000,9.830396,1.062500
10,13.350800,7.267451,4.571300,1.313000,0.656000,9.736973,1.125000
12,11.197400,6.760006,4.592300,1.307000,0.653000,10.080625,0.953125


tensor(13.8090, device='cuda:0', grad_fn=<MulBackward0>)
tensor(3.0077, device='cuda:0')
tensor(4.2657, device='cuda:0')
tensor(5.8103, device='cuda:0')
tensor(10.9284, device='cuda:0', grad_fn=<MulBackward0>)
tensor(15.6959, device='cuda:0', grad_fn=<MulBackward0>)
tensor(9.3948, device='cuda:0')
tensor(11.2430, device='cuda:0')
tensor(13.1583, device='cuda:0')
tensor(13.4684, device='cuda:0', grad_fn=<MulBackward0>)
tensor(14.8901, device='cuda:0', grad_fn=<MulBackward0>)
tensor(6.6060, device='cuda:0')
tensor(7.1291, device='cuda:0')
tensor(13.1895, device='cuda:0')
tensor(18.1063, device='cuda:0', grad_fn=<MulBackward0>)
tensor(10.5454, device='cuda:0', grad_fn=<MulBackward0>)
tensor(5.6300, device='cuda:0')
tensor(7.3976, device='cuda:0')
tensor(9.8304, device='cuda:0')
tensor(14.1671, device='cuda:0', grad_fn=<MulBackward0>)
tensor(13.3508, device='cuda:0', grad_fn=<MulBackward0>)
tensor(4.9363, device='cuda:0')
tensor(7.1291, device='cuda:0')
tensor(9.7370, device='cuda:0')
tens

KeyboardInterrupt: 

In [36]:
trainer.state.distill_loss = 5

In [38]:
trainer.state.distill_loss

5

In [17]:
outputs.predictions.shape

(54, 32, 50257)

In [19]:
logits = torch.rand(5000, 1024, 50257)

In [ ]:
raw_datase

In [97]:
outputs.predictions[0].shape

(32, 50257)

In [98]:
out = model.forward(torch.tensor([[0, 1]]))

In [101]:
out.logits.shape

torch.Size([1, 2, 50257])

In [63]:
logits[0][0].shape

(12, 8, 32, 64)

In [45]:
logits[0][0].shape

(12, 8, 32, 64)

In [49]:
len(eval_dataset[0]['input_ids'])

32

In [53]:
eval_dataset[1]

{'input_ids': [247,
  8230,
  281,
  253,
  2798,
  273,
  27015,
  41151,
  23749,
  1157,
  253,
  2846,
  1907,
  644,
  387,
  253,
  43213,
  273,
  1302,
  314,
  1495,
  1157,
  1445,
  285,
  10336,
  323,
  689,
  247,
  8014,
  1107,
  1157,
  342],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [247,
  8230,
  281,
  253,
  2798,
  273,
  27015,
  41151,
  23749,
  1157,
  253,
  2846,
  1907,
  644,
  387,
  253,
  43213,
  273,
  1302,
  314,
  1495,
  1157,
  1445,
  285,
  10336,
  323,
  689,
  247,
  8014,
  1107,
  1157,
  342]}

In [72]:
logits[1][1].shape

(12, 8, 32, 64)

In [74]:
64 * 8 * 12

6144

In [73]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [ ]:

def run_train(
    model_args,
    data_args,
    training_args,
    config,
):
    
    # Load pretrained model
    # if config.model_type == 'Llama':
    #     model_type = LlamaForCausalLM
    # else:
    #     model_type = AutoModelForCausalLM
    

    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        torch_dtype=torch.bfloat16,
        token=model_args.token,
        cache_dir=model_args.cache_dir,
        device_map="auto"	    
)
    
    if config.zero_outliers:
        make_zero_outliers(model, config.outlier_fraction)

    if config.use_clip_softmax:
        model.set_clipped_sm(gamma=config.clip_softmax_gamma, eta=config.clip_softmax_eta)

    if config.ste.enable:
        outlier_ids, layer_bit = prepare_llama_ste(config.ste.path_to_act_scales, config.ste.fp_features_num, **config.ste.layer_bits)
        model.enable_ste(outlier_ids=outlier_ids, layer_bit=layer_bit, block_size=config.ste.block_size)

    if config.use_lora:
        task_type = TaskType.CAUSAL_LM
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"]
        lora_config = LoraConfig(
            task_type=task_type,
            inference_mode=False,
            r=config.lora_rank,
            lora_alpha=config.lora_alpha,
            lora_dropout=config.lora_dropout,
            target_modules=config.lora_target_modules,
            init_lora_weights=True,
        )
        model = get_peft_model(model, lora_config)

    # Load pretrained tokenizer
    tokenizer_kwargs = {
        "cache_dir": model_args.cache_dir,
        "use_fast": model_args.use_fast_tokenizer,
        "revision": model_args.model_revision,
        "token": model_args.token,
        "trust_remote_code": model_args.trust_remote_code,
    }

    if model_args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
    elif model_args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)

    #Load and preprocessing dataset
    raw_datasets = load_hf_datasets(data_args)
    tokenized_datasets = tokenize_datasets(data_args, raw_datasets, tokenizer)
    lm_datasets = format_datasets(data_args, tokenized_datasets, tokenizer)

    data_collator = DataCollatorWithMaskForCausalLM(
        tokenizer=tokenizer
    )

    if config.norm_tweek:
        layernorm_names = [f"model.layers.{layer_block_num}.input_layernorm.weight" for layer_block_num in range(len(model.model.layers))]
        layernorm_names += [f"model.layers.{layer_block_num}.post_attention_layernorm.weight" for layer_block_num in range(len(model.model.layers))]

        #Set model parameters to be learned
        for name, param in model.named_parameters():
            if name not in layernorm_names:
                # freeze base model's layers
                param.requires_grad = False
            else:
                param.requires_grad = True

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable_params: {trainable_params}")

    
    #Train
    train_dataset = lm_datasets["train"]
    eval_dataset = lm_datasets["validation"]

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        # Data collator will default to DataCollatorWithPadding, so we change it.
        data_collator=default_data_collator
    )

    trainer.save_model()
    train_result = trainer.train()
    trainer.save_model()  # Saves the tokenizer too for easy upload

